<a href="https://colab.research.google.com/github/Aviawesome/parrallelcomputing/blob/main/practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%writefile arrsum.cpp

#include<iostream>
#include<vector>
#include<mpi.h>
#include<cstdlib>
#define N 20
using namespace std;
int main(int argc, char **argv)
{
    vector<int> v,v1;
 int rank,size;
 int n,a,sum=0,b,c;
  MPI_Init(&argc,&argv);
 MPI_Comm_size(MPI_COMM_WORLD,&size);
 MPI_Comm_rank(MPI_COMM_WORLD,&rank);
 srand(time(NULL)+rank);
 for(int i=0;i<N;i++)
 {
     c=rand()%100+1;
     v.push_back(c);
 }
 a=N/size;
 for(int i=rank*a;i<(rank+1)*a;i++)
 {
     sum+=v[i];
 }
 if(rank==(size-1))
 {
     if(N%size!=0)
     {
         a=N%size;
      for(int i=N-1;i>=N-a;i--)
      {
          sum=sum+v[i];
      }
     }
 }
 cout<<"sum="<<sum<<" of rank "<<rank<<"\n";
 if(rank==0)
 {
     for(int i=1;i<size;i++)
     {
         MPI_Recv(&b,1,MPI_INT,i,0,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
      v1.push_back(b);
     }
 }
 else
 {
     MPI_Send(&sum,1,MPI_INT,0,0,MPI_COMM_WORLD);
 }
 MPI_Barrier(MPI_COMM_WORLD);
 if(rank==0)
 {
     for(int i=0;i<size-1;i++)
     {
         sum=sum+v1[i];
     }
  cout<<"sum="<<sum;
 }
 MPI_Finalize();
 return 0;
}

Overwriting arrsum.cpp


In [9]:
!mpic++ arrsum.cpp
!mpirun -np 2 --allow-run-as-root ./a.out

sum=474 of rank 1
sum=537 of rank 0
sum=1011

In [12]:
%%writefile arrsum1.cpp

#include<iostream>
#include<vector>
#include<mpi.h>
#include<cstdlib>
#define N 20
using namespace std;
int main(int argc, char **argv)
{
    vector<int> v,v1;
 int rank,size;
 int n,a,sum=0,b,c;
  MPI_Init(&argc,&argv);
 MPI_Comm_size(MPI_COMM_WORLD,&size);
 MPI_Comm_rank(MPI_COMM_WORLD,&rank);
 a=N/size;
 for(int i=rank*a;i<(rank+1)*a;i++)
 {
     v.push_back(i);
 }
 if(rank==(size-1))
 {
     if(N%size!=0)
     {
         a=N%size;
      for(int i=N-1;i>=N-a;i--)
      {
          sum=sum+v[i];
      }
     }
 }
  for(int i=0;i<v.size();i++)
 {
     sum=sum+v[i];;
 }
 cout<<"sum="<<sum<<" of rank "<<rank<<"\n";
 if(rank==0)
 {
     for(int i=1;i<size;i++)
     {
         MPI_Recv(&b,1,MPI_INT,i,0,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
      v1.push_back(b);
     }
 }
 else
 {
     MPI_Send(&sum,1,MPI_INT,0,0,MPI_COMM_WORLD);
 }
 MPI_Barrier(MPI_COMM_WORLD);
 if(rank==0)
 {
     for(int i=0;i<size-1;i++)
     {
         sum=sum+v1[i];
     }
  cout<<"sum="<<sum;
 }
 MPI_Finalize();
 return 0;
}

Overwriting arrsum1.cpp


In [13]:
!mpic++ arrsum1.cpp
!mpirun -np 4 --allow-run-as-root ./a.out

sum=35 of rank 1
sum=85 of rank 3
sum=10 of rank 0
sum=60 of rank 2
sum=190

In [27]:
%%writefile exchange.cpp


#include<iostream>
#include<vector>
#include<mpi.h>
#include<cstdlib>
using namespace std;
int main(int argc, char **argv)
{
  int v[10],v1[10];
 int *p,*q;
 int rank,size;
 int n,a,b,c;
 p=v;
 q=v1;
  MPI_Init(&argc,&argv);
 MPI_Comm_size(MPI_COMM_WORLD,&size);
 MPI_Comm_rank(MPI_COMM_WORLD,&rank);
 n=10;
 srand(time(NULL)+rank);
 for(int i=0;i<n;i++)
 {
     a=rand();
  v[i]=a;
 }
 cout<<"rank "<<rank<<" value before exchange is->";
 for(int i=0;i<n;i++)
 {
     cout<<v[i]<<" ";
 }
 cout<<"\n";
if(rank<size)
{
    b=rank;
 b=(rank+1)%size;
 MPI_Send(&v,n,MPI_INT,b,0,MPI_COMM_WORLD);
 c=(rank+size-1)%size;
 MPI_Recv(&v1,n,MPI_INT,c,0,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
 for(int i=0;i<n;i++)
 {
     v[i]=v1[i];
 }
}
 MPI_Barrier(MPI_COMM_WORLD);
 cout<<"rank "<<rank<<" value after exchange is->";
 for(int i=0;i<n;i++)
 {
     cout<<v[i]<<" ";
 }
 cout<<"\n";
 MPI_Finalize();
 return 0;
}

Overwriting exchange.cpp


In [28]:
!mpic++ exchange.cpp
!mpirun -np 2 --allow-run-as-root ./a.out

rank 0 value before exchange is->780875042 91209273 431103531 955054008 1481415418 231461206 1803357862 1803063807 2062990834 1517871801 
rank 1 value before exchange is->1546085972 973197252 1078989240 563792767 1333467057 703101333 1499369200 281449496 1642013167 619797117 
rank 0 value after exchange is->1546085972 973197252 1078989240 563792767 1333467057 703101333 1499369200 281449496 1642013167 619797117 
rank 1 value after exchange is->780875042 91209273 431103531 955054008 1481415418 231461206 1803357862 1803063807 2062990834 1517871801 


In [13]:
%%writefile vector.cpp



#include<iostream>
#include<vector>
#include<mpi.h>
#include<cstdlib>
#define N 20
using namespace std;
int main(int argc, char **argv)
{
  vector<int64_t> v,v1,v2;
 int rank,size,c,n,a;
 int64_t b,sum=0;;
  MPI_Init(&argc,&argv);
 MPI_Comm_size(MPI_COMM_WORLD,&size);
 MPI_Comm_rank(MPI_COMM_WORLD,&rank);
  n=N/size;
 srand(time(NULL)+rank);
 for(int i=(rank*n);i<(rank+1)*n;i++)
 {
     c=rand()%10+1;
     v.push_back(c);
 }
 for(int i=(rank*n);i<(rank+1)*n;i++)
 {
     c=rand()%10+1;
     v1.push_back(c);   
 }
 if(rank==(size-1))
 {
     if(N%size!=0)
     {
         a=N%size;
      for(int i=N-1;i>=N-a;i--)
      {
          c=rand()%10+1;
          v.push_back(c);
      }
            for(int i=N-1;i>=N-a;i--)
      {
          c=rand()%10+1;
          v1.push_back(c);
      }
     }
 }
 for(int i=0;i<v.size();i++)
 {
     cout<<v[i]<<" ";
 }
 cout<<"\n";
  for(int i=0;i<v1.size();i++)
 {
     cout<<v1[i]<<" ";
 }
 cout<<"\n";
  for(int i=0;i<v.size();i++)
 {
     c=v[i]*v1[i];
     sum=sum+c;
 }
 MPI_Barrier(MPI_COMM_WORLD);
 cout<<"rank is "<<rank<<" sum is "<<sum<<"\n";
  if(rank==0)
 {
     for(int i=1;i<size;i++)
     {
         MPI_Recv(&a,1,MPI_INT,i,0,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
      v2.push_back(a);
     }
 }
 else
 {
     MPI_Send(&sum,1,MPI_INT,0,0,MPI_COMM_WORLD);
 }
 MPI_Barrier(MPI_COMM_WORLD);
 if(rank==0)
 {
          for(int i=0;i<size-1;i++)
     {
         sum=sum+v2[i];
     }
  cout<<"sum= "<<sum;
 }
 MPI_Finalize();
 return 0;
}

Overwriting vector.cpp


In [14]:
!mpic++ vector.cpp
!mpirun -np 8 --allow-run-as-root ./a.out

10 8 
2 4 
6 3 
7 5 
4 4 
5 8 
7 8 
1 4 
7 10 
6 7 
3 8 4 5 5 3 
9 1 1 1 6 6 
7 7 
5 9 
7 3 
9 4 
rank is 4 sum is 52
rank is 5 sum is 75
rank is 1 sum is 112
rank is 6 sum is 39
rank is 2 sum is 57
rank is 0 sum is 52
rank is 7 sum is 92
rank is 3 sum is 98
sum= 577